In [4]:
import pandas
from matplotlib import pyplot
from mlflow import set_tracking_uri
import json
from typing import Dict

from hcve_lib.tracking import get_children_runs
from hcve_lib.tracking import get_completed_runs
from hcve_lib.tracking import is_root_run
from hcve_lib.visualisation import h3, h1, h4

set_tracking_uri('http://localhost:5000')

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
pandas.set_option("display.max_columns", None)
pyplot.rcParams['figure.facecolor'] = 'white'

def get_lco_hyperparameters(method_name: str) -> Dict[str, Dict]:
    runs_lco = get_completed_runs('optimized_lco')
    root_lco_runs = runs_lco[is_root_run(runs_lco)]
    coxnet_run = root_lco_runs.query(f'`tags.mlflow.runName` == "{method_name}"').iloc[0]
    children_runs = get_children_runs(runs_lco, coxnet_run['run_id'])
    return {
        rows['tags.mlflow.runName']: json.loads(rows['params.hyperparameters'].replace("\'", "\""))
        for _, rows in children_runs.iterrows()
    }
